In [1]:
import os
os.chdir('..')

In [2]:
import torch
from torch import nn
import numpy as np
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from model.m2m_CVAE import CVAE

In [3]:
## Loss function
def loss_fn(loss_function, logp, target, length, mean, log_var, anneal_function, step, k, x0):

    # Negative Log Likelihood
    NLL_loss = loss_function(logp, target)

    # KL Divergence
    KL_loss = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    KL_weight = kl_anneal_function(anneal_function, step, k, x0)

    return NLL_loss, KL_loss, KL_weight

## Annealing function 
def kl_anneal_function(anneal_function, step, k, x0):
        if anneal_function == 'logistic':
            return float(1/(1+np.exp(-k*(step-x0))))
        elif anneal_function == 'linear':
            return min(1, step/x0) 

In [7]:
step = 1
batch_size = 5

device = 'cpu'

# Annealing parameter
k = 0.0025
x0 = 2500

loss_function = torch.nn.NLLLoss(reduction='sum')

model = CVAE().to(device)

melody = torch.randn(batch_size, 272, 12 * 24 * 2)
chord_onehot = torch.randn(batch_size, 272, 96)
length = torch.Tensor([272, 100, 200, 150, 120]).long()

melody_pred, logp ,mu, log_var, _ = model(chord_onehot,melody,length)

# Arrange 
pred_flatten = []
groundtruth_flatten = []
logp_flatten = []
length = length.squeeze()

for i in range(batch_size):

    # Get predicted softmax chords by length of the song (cutting off padding 0), (1,length,96)
    logp_flatten.append(logp[i][:length[i]])

    # Get predicted softmax chords by length of the song (cutting off padding 0), (1,length,12 * 24 * 2)
    pred_flatten.append(melody_pred[i][:length[i]])

    # Get groundtruth chords by length of the song (cutting off padding 0), (1,length)
    groundtruth_flatten.append(melody[i][:length[i]])

# Rearrange for loss calculatio
logp_flatten = torch.cat(logp_flatten, dim=0)
pred_flatten = torch.cat(pred_flatten, dim=0)
groundtruth_flatten = torch.cat(groundtruth_flatten,dim=0).long()
groundtruth_index = torch.max(groundtruth_flatten,1).indices

# loss calculation
# Add weight to NLL also
NLL_loss, KL_loss, KL_weight = loss_fn(loss_function = loss_function, logp = logp_flatten, target = groundtruth_index, length = length, mean = mu, log_var = log_var,anneal_function='logistic', step=step, k=k, x0=x0)
# NLL_loss, KL_loss, KL_weight = loss_fn(logp = chord_pred_flatten, target = chord_groundtruth_index, length = length, mean = mu, log_var = log_var,anneal_function='logistic', step=step, k=k, x0=x0)

loss = (NLL_loss + KL_weight * KL_loss) / batch_size
print('loss: ',loss.item())


loss:  1080.3507080078125
